# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [2]:
import string
import re
import collections
from collections import Counter
import nltk
from sklearn.base import TransformerMixin
import numpy as np

In [3]:
text = open('peace.txt', 'r', encoding='utf-8-sig').read()
len(text)

3227580

In [4]:
def preprocess_text(text):
    punct = list(set(re.findall('\W', text)))
    punct.remove('.')
    words = ''
    for word in text.split():
        for letter in word:
            if letter not in punct:
                words += letter
        words += ' '
    text = words.lower()
        
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    return text


text = preprocess_text(text)
len(text)
# assert len(text) == 3141169

3130569

In [5]:
text = text.split('.')
text = [x.strip() for x in text]

In [6]:
class BPE(TransformerMixin):
    
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set([item for txt in text for item in set(txt)]))
        self.stoi = {l: idx for idx, l in enumerate(self.itos)}
        text = [[self.stoi[letter] for letter in txt] for txt in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigrams = collections.Counter()
            for txt in text:
                i = 0
                while i + 1 < len(txt):
                    bigrams[(txt[i], txt[i+1])] += 1
                    i += 1
                
                
            mc = bigrams.most_common(1)[0][0]
            new_token = str(self.itos[int(mc[0])]) + str(self.itos[int(mc[1])])
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            tmp = []
            for txt in text:
                tmp2 = []
                i = 0
                while i + 1 < len(txt):
                    if new_token == self.itos[txt[i]] + self.itos[txt[i+1]]:
                        tmp2.append(new_id)
                        i += 2
                    else:
                        tmp2.append(txt[i])
                        i += 1
                tmp.append(tmp2)
                    
            text = tmp 
           
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """ 
        max_size = max([len(tok) for tok in self.itos])
        
        new_text = []
        for txt in text:
            i = 0
            new_txt = []
            while i < len(txt):
                hit = False
                stop = i+max_size if len(txt) - (i + max_size) >= 0 else len(txt)
                while hit == False and stop > i:
                    if txt[i:stop] in self.itos:
                        new_txt.append(self.stoi[txt[i:stop]])
                        hit = True
                        i = stop
                    else:
                        stop -= 1
            new_text.append(new_txt)
                                
        text = new_text
       # for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
          #  text = # TODO       
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = self.itos[tok] if isinstance(tok, int) else [self.itos[i] for i in token]
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [7]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [8]:
bpe.decode(tokenized_text[0]) 

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever'

In [9]:
text[0]

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever'

In [10]:
print(tokenized_text[0])

[65, 49, 47, 50, 8, 26, 10, 61, 29, 14, 55, 69, 20, 64, 73, 26, 20, 50, 50, 52, 0, 83, 3, 82, 0, 71, 49, 26, 35, 10, 57, 20, 66, 38, 26, 68, 55, 50, 38, 89, 50, 66, 58, 6, 60, 26, 20, 50, 50, 52, 0, 6, 60, 44, 75, 0, 65, 14, 53, 57, 83, 63, 21, 67, 57, 63, 21, 3, 28, 64, 57, 84, 19, 68, 10, 50, 89, 0, 71, 99, 58, 0, 87, 54, 50, 89, 0, 19, 68, 79, 89, 47, 6, 10, 55, 6, 67, 60, 3, 78, 55, 53, 50, 26, 9, 64]


In [48]:
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = Counter() # TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = {}
        for item in self.itos:
            result[item] = self.get_proba(a, b, item, tau)
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        # TODO approximate probability by counters
        
        P_abc = self.proba[a, b, c] + self.delta
        ab = [item for item in self.proba.keys() if item[0] == a and item[1] == b]
        P_ab = sum([self.proba[item] for item in ab]) + self.delta
        n = 1/tau        
        result = (P_abc ** n) / (P_ab ** n) if P_abc > 0 else 0 
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        self.text = text
        for txt in text:
            for i in range(len(txt) - 2):
                self.proba[txt[i], txt[i+1], txt[i+2]] += 1
                                       
        # self.proba = # TODO count 3-grams in the text
    
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

lm.get_proba(65, 49, 47)

0.31179185900125894

In [55]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    # TODO store in beam tuples of current sequences and their log probabilities
    
    if len(input_seq) >= 2:
        beam = [item for item in lm.proba.keys() if item[0] == input_seq[-2] and item[1] == input_seq[-1]]
        beam = {item: lm.get_proba(item[0], item[1], item[2]) for item in beam}
    else:
        beam = [item for item in lm.proba.keys() if item[0] == input_seq[0]]
        beam = {item: lm.get_proba(item[0], item[1], item[2])  for item in beam}
    
    print(beam)
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
       # for snt, snt_proba in beam:
       #     if # TODO process terminal token
       #     else:    
       #         proba = # probability vector of the next token
       #         best_k = # top-k most probable tokens
                # TODO update candidates' sequences and corresponding probabilities
                
       # beam = # select top-k most probable sequences from candidates
    return beam

In [56]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
    

{(53, 57, 83): 0.07580477673935618, (53, 57, 6): 0.04132917964693666, (53, 57, 62): 0.006853582554517134, (53, 57, 71): 0.06770508826583593, (53, 57, 3): 0.09221183800623053, (53, 57, 44): 0.033229491173416406, (53, 57, 78): 0.007892004153686396, (53, 57, 1): 0.02679127725856698, (53, 57, 54): 0.0367601246105919, (53, 57, 21): 0.012876427829698858, (53, 57, 29): 0.017445482866043614, (53, 57, 99): 0.016822429906542057, (53, 57, 79): 0.009761163032191069, (53, 57, 90): 0.024299065420560748, (53, 57, 35): 0.01806853582554517, (53, 57, 58): 0.03842159916926272, (53, 57, 10): 0.02679127725856698, (53, 57, 49): 0.03842159916926272, (53, 57, 81): 0.03509865005192108, (53, 57, 26): 0.01391484942886812, (53, 57, 86): 0.037383177570093455, (53, 57, 28): 0.011214953271028037, (53, 57, 9): 0.008515057113187955, (53, 57, 65): 0.025129802699896158, (53, 57, 53): 0.0494288681204569, (53, 57, 20): 0.03489096573208723, (53, 57, 19): 0.01973001038421599, (53, 57, 55): 0.02118380062305296, (53, 57, 47):

In [ ]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [ ]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities

In [ ]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

In [ ]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    result = #TODO perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)